In [ ]:
import os
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import matplotlib
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/figures/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

dn = smooth.analysis.get_display_names()

In [ ]:
LOG_DIR = "/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/"

ms  = pd.read_feather(os.path.join(LOG_DIR, "0326_mnist_binary/measures.feather"))
ms2 = pd.read_feather(os.path.join(LOG_DIR, "0329_mnist_binary_gradient_norm/measures.feather"))
ms3 = pd.read_feather(os.path.join(LOG_DIR, "0331_mnist_binary_weights_product/measures.feather"))

ms = pd.concat([ms, ms2, ms3], sort=False)
ms = ms.reset_index(drop=True)

print("Removing {} entries".format(sum(ms["gradient_norm_test"].isna())))
ms = ms[~ms["gradient_norm_test"].isna()]
ms["model.weights_product_reg_coef"] = ms["model.weights_product_reg_coef"].fillna(value=0)

smooth.analysis.remove_constant_columns(ms, verbose=True)

ms["kind"] = "normal"
for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.weights_product_reg_coef"] == val, "kind"] = "weights_product_{:.0e}".format(val)
# ms.loc[ms["model.weights_product_reg_coef"] > 0, "kind"] = "weights_product"

for val in [1e-5, 1e-4, 1e-3]:
    ms.loc[ms["model.gradient_norm_reg_coef"] == val, "kind"] = "gradient_norm_{:.0e}".format(val)

# Add normalized gradient norm
ms1 = ms[ms["kind"] == "normal"].copy()
ms1 = ms1.rename(columns={"gradient_norm_test": "gradient_norm_test_unreg"})
ms = pd.merge(
    ms,
    ms1[["dataset.name", "model.hidden_size", "gradient_norm_test_unreg"]],
    on=["dataset.name", "model.hidden_size"],
)

ms["gradient_norm_test_normalized"] = ms["gradient_norm_test"] / ms["gradient_norm_test_unreg"]
del ms["gradient_norm_test_unreg"]

ms

In [ ]:
def plot_measure(ms1, measure_name, hue_column):
    ms1 = ms1.rename(columns=dn)
    
    grid = sns.lineplot(
        data=ms1,
        x=dn["model.hidden_size"],
        y=dn[measure_name],
        hue=dn[measure_name],
        palette=smooth.analysis.make_palette(ms1[dn[measure_name]]),
#         col="kind",
#         hue_order=sorted(ms1["kind"].unique()),
#         kind="line",
    )
    ax = grid
#     ax = grid.axes[0][0]
    if "loss" in measure_name:
        ax.set_yscale("log")
    ax.set_xscale("log")
    ax.set_xlim((3, 1200))
    ax.set_xticks(sorted(ms["model.hidden_size"].unique()))
#     ax.get_xaxis().get_major_formatter().set_scientific(False)
    ax.xaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())

    for t in ax.get_legend().texts[1:]:
#         print(t.get_text())
        t.set_text("${}$".format(smooth.analysis.to_scientific_tex(float(t.get_text()))))
    plt.show()

In [ ]:
ms1 = ms[ms["model.weights_product_reg_coef"] == 0]

for measure in [
            "loss_train",
            "loss_test",
            "gradient_norm_train",
            "gradient_norm_test",
            "gradient_norm_test_normalized",
#             "weights_product",
        ]:
    plt.figure()
    plot_measure(ms1[:10], measure, "model.gradient_norm_reg_coef")
#     plt.savefig("gradient_norm_reg_{}.pdf".format(measure), bbox_inches="tight")

In [ ]:
ms1 = ms[ms["model.gradient_norm_reg_coef"] == 0]

for measure in [
            "loss_train",
            "loss_test",
            "weights_product",
#             "gradient_norm_test_normalized",
#             "weights_product",
        ]:
    plt.figure()
    plot_measure(ms1[:10], measure, "model.weights_product_reg_coef")
#     plt.savefig("gradient_norm_reg_{}.pdf".format(measure), bbox_inches="tight")